###  STEP 1: Documents Preparation

In [7]:
import json

with open("../data/documents.json","rt") as file_in:
    raw_document = json.load(file_in)

In [21]:
information = []


for course in raw_document:
    info = {}
    info['course'] = course['course']
    info['questions'] = len(course['documents'])
    information.append(info)


information

[{'course': 'data-engineering-zoomcamp', 'questions': 435},
 {'course': 'machine-learning-zoomcamp', 'questions': 375},
 {'course': 'mlops-zoomcamp', 'questions': 138}]

In [23]:
documents = []


for course in raw_document:
    for doc in course['documents']:
        doc['course'] = course['course']
        documents.append(doc)

In [25]:
len(documents)

948

In [26]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

###  STEP 2: Create Embedings using Pretrained Models

In [31]:
from sentence_transformers import SentenceTransformer


model = SentenceTransformer("all-mpnet-base-v2")

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [33]:
len(model.encode("my name is mide"))

768

In [34]:
embeded_documents = []

for doc in documents:
    doc['text_vector'] = model.encode(doc['text']).tolist()
    embeded_documents.append(doc)

embeded_documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'text_vector': [-0.035703498870134354,
  -0.06891412287950516,
  -0.044483356177806854,
  0.00625010346993804,
  -0.04224717244505882,
  -0.006454021669924259,
  0.027548067271709442,
  -0.01861034892499447,
  -0.05203762650489807,
  -0.002404673956334591,
  0.057454537600278854,
  -0.015592606738209724,
  0.01548878476023674,
  0.09870165586471558,
  0.015649670735001564,
  -0.

### STEP 3: Setup ElasticSearch Connection 

In [13]:
from elasticsearch import Elasticsearch

#e_client = Elasticsearch('http://localhost:9200')

e_client = Elasticsearch('https://a87e40b7c8747423f8a29589725d1ff6-1251252454.eu-central-1.elb.amazonaws.com:9200')

e_client.info()

/Users/adebimpe/Desktop/llm_zoomcamp/llm_env/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'a87e40b7c8747423f8a29589725d1ff6-1251252454.eu-central-1.elb.amazonaws.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


ObjectApiResponse({'name': 'elasticsearch-client-0', 'cluster_name': 'elasticsearch', 'cluster_uuid': '8QSqN2L5S5-SP-korInjmQ', 'version': {'number': '8.5.1', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'c1310c45fc534583afe2c1c03046491efba2bba2', 'build_date': '2022-11-09T21:02:20.169855900Z', 'build_snapshot': False, 'lucene_version': '9.4.1', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

### STEP 4: Create Mapping and Index

In [38]:
index_name = "course-questions"

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "text_vector": {"type": "dense_vector","dims": 768,"index": True,"similarity":"cosine"},


        }
    }
}

e_client.indices.delete(index=index_name,ignore_unavailable=True)
e_client.indices.create(index=index_name,body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

### STEP 5:  Add documents into Index

In [39]:
for doc in embeded_documents:
    try: 
       e_client.index(index=index_name,document=doc)
    except Exception as e:
        print(e)

### STEP 6: Create end user query

In [40]:
search_term = "windows or mac?"
vector_search_term = model.encode(search_term)

In [41]:
query = {
    "field" : "text_vector",
    "query_vector" : vector_search_term,
    "k" : 5,
    "num_candidates" : 10000,
}

In [42]:
res = e_client.search(index=index_name,knn=query,source=["text","section","question","course"])

res["hits"]["hits"]


[{'_index': 'course-questions',
  '_id': 'SUR8jJIBJvrv-SN4fYVV',
  '_score': 0.71479183,
  '_source': {'question': 'Environment - Is the course [Windows/mac/Linux/...] friendly?',
   'course': 'data-engineering-zoomcamp',
   'section': 'General course-related questions',
   'text': 'Yes! Linux is ideal but technically it should not matter. Students last year used all 3 OSes successfully'}},
 {'_index': 'course-questions',
  '_id': 'XER8jJIBJvrv-SN4iIiE',
  '_score': 0.61347353,
  '_source': {'question': 'WSL instructions',
   'course': 'mlops-zoomcamp',
   'section': 'Module 1: Introduction',
   'text': 'If you wish to use WSL on your windows machine, here are the setup instructions:\nCommand: Sudo apt install wget\nGet Anaconda download address here. wget <download address>\nTurn on Docker Desktop WFree Download | AnacondaSL2\nCommand: git clone <github repository address>\nVSCODE on WSL\nJupyter: pip3 install jupyter\nAdded by Gregory Morris (gwm1980@gmail.com)\nAll in all softwares 

### STEP 7: Perform keyword search with Sematic & advanced search

In [43]:
a_res = e_client.search(index=index_name,
                        knn=query,
                        query= { "match" : {"course": "data-engineering-zoomcamp"}},
                        source=["text","section","question","course"]
                        )

a_res["hits"]["hits"]

[{'_index': 'course-questions',
  '_id': 'SUR8jJIBJvrv-SN4fYVV',
  '_score': 1.4937057,
  '_source': {'question': 'Environment - Is the course [Windows/mac/Linux/...] friendly?',
   'course': 'data-engineering-zoomcamp',
   'section': 'General course-related questions',
   'text': 'Yes! Linux is ideal but technically it should not matter. Students last year used all 3 OSes successfully'}},
 {'_index': 'course-questions',
  '_id': 'OkR8jJIBJvrv-SN4fYUL',
  '_score': 1.3775007,
  '_source': {'question': 'Environment - Should I use my local machine, GCP, or GitHub Codespaces for my environment?',
   'course': 'data-engineering-zoomcamp',
   'section': 'General course-related questions',
   'text': 'You can set it up on your laptop or PC if you prefer to work locally from your laptop or PC.\nYou might face some challenges, especially for Windows users. If you face cnd2\nIf you prefer to work on the local machine, you may start with the week 1 Introduction to Docker and follow through.\nHow